In [ ]:

import pandas as pd

# Add parent directory to path to import local modules
# sys.path.insert(0, r"C:/Users/sfgar/programing/INTED-article-splitting/__computational_essays/python_code")

from LLM_classifier import (
    load_categories_from_json
)
from theory_classifiers import TheoreticalFrameworkClassifier
from ..python_code import helpers


from LLM_classifier.cost_estimator import CostEstimationWrapper


ModuleNotFoundError: No module named 'section_classifier'

In [7]:

api_key = helpers.safe_load_env_variable("OPENAI_API_KEY")
# Initialize the classifier with loaded frameworks
classifier_mini = TheoreticalFrameworkClassifier(
    api_key = api_key,
    temperature=0.3,
    general_model="gpt-4.1-mini",
    reasoning_model="o3-mini"
)


classifier_normal = TheoreticalFrameworkClassifier(
    api_key = api_key,
    temperature=0.3,
    general_model="gpt-4.1",
    reasoning_model="o3-mini"
)

In [8]:

# Load the data
df = pd.read_pickle("C:/Users/sfgar/programing/INTED-article-splitting/__computational_essays/data/classified_sections_light_gpt-4.1-mini_20250716_122412.pkl")
frameworks = load_categories_from_json("./reviewed_categories_20250721_163329.json")

Successfully loaded 135 categories from ./reviewed_categories_20250721_163329.json


In [9]:

# Helper function to filter DataFrame by category
def filter_df_by_category(
    df,
    value: str,
    column: str = "classification_highest_prob_gpt-4.1-mini"
):
    """
    Returns a DataFrame filtered to only rows where the specified column matches the given value.
    """
    mask = df[column] == value
    return df[mask].copy()

# Filter for theoretical framework sections
theory_df = filter_df_by_category(df, "Theoretical Framework")
methods_df = filter_df_by_category(df, "Methods")

In [10]:
cost_estimator_mini = CostEstimationWrapper(classifier_mini)
theory_data = classifier_mini._df_to_sections(theory_df)
theory_preped_discovery_data = [section.text for section in theory_data]
methods_data = classifier_mini._df_to_sections(methods_df)
methods_preped_discovery_data = [section.text for section in methods_data]
all_data = classifier_mini._df_to_sections(df)

section_categories = load_categories_from_json("C:/Users/sfgar/programing/INTED-article-splitting/__computational_essays/data/classifiers_LLM_generated.json")


Successfully loaded 13 categories from C:/Users/sfgar/programing/INTED-article-splitting/__computational_essays/data/classifiers_LLM_generated.json


In [11]:

theory_est_mini = cost_estimator_mini.estimate_full_workflow(theory_data, frameworks, theory_preped_discovery_data)
method_est_mini = cost_estimator_mini.estimate_full_workflow(methods_data, frameworks, methods_preped_discovery_data)
all_mini = cost_estimator_mini.estimate_full_workflow(all_data, section_categories)

In [12]:


cost_estimator_normal = CostEstimationWrapper(classifier_normal)

theory_est_normal = cost_estimator_normal.estimate_full_workflow(theory_data, frameworks, theory_preped_discovery_data)
method_est_normal = cost_estimator_normal.estimate_full_workflow(methods_data, frameworks, methods_preped_discovery_data)
all_est_normal = cost_estimator_normal.estimate_full_workflow(all_data, section_categories)




In [38]:

data = {
    "Step label": ["Discovery", "Classification", "Review", "Classification"], 
    'Methods identification': [method_est_mini["category_discovery"].total_cost, method_est_mini["batch_classification"].total_cost, method_est_mini["category_review"].total_cost, method_est_mini["batch_classification"].total_cost],
    'Theory identification': [None,theory_est_mini["batch_classification"].total_cost,None,None],
    # 'Section classification': [0,all_mini["batch_classification"].total_cost,0,0]
}

cost_df = pd.DataFrame(data)

cost_df.loc['Total'] = cost_df.sum(numeric_only=True)

price_columns = [x for x in cost_df.keys()if x != "Step label"]

cost_df.style.set_properties(**{'text-align': 'center', 'background-color': '#e9f5e9'}).format('${:,.2f}', subset=price_columns).set_caption("Run costs with gpt-4.1-mini").apply(lambda x: ['font-weight: bold'] * len(x), axis=1, subset=pd.IndexSlice['Total', :])


,Step label,Methods identification,Theory identification
0,Discovery,$0.04,$nan
1,Classification,$5.77,$2.13
2,Review,$0.00,$nan
3,Classification,$5.77,$nan
Total,nan,$11.59,$2.13


In [39]:


data = {
    "Step label": ["Discovery", "Classification", "Review", "Classification"], 
    'Methods identification': [method_est_normal["category_discovery"].total_cost, method_est_normal["batch_classification"].total_cost, method_est_normal["category_review"].total_cost, method_est_normal["batch_classification"].total_cost],
    'Theory identification': [None,theory_est_normal["batch_classification"].total_cost,None,None],
    "Section classification": [None, all_est_normal["batch_classification"].total_cost,None,None]
}

cost_df = pd.DataFrame(data)

cost_df.loc['Total'] = cost_df.sum(numeric_only=True)

price_columns = [x for x in cost_df.keys()if x != "Step label"]

cost_df.style.set_properties(**{'text-align': 'center', 'background-color': '#FFE5E5'}).format('${:,.2f}', subset=price_columns).set_caption("Run costs with gpt-4.1").apply(lambda x: ['font-weight: bold'] * len(x), axis=1, subset=pd.IndexSlice['Total', :])

,Step label,Methods identification,Theory identification,Section classification
0,Discovery,$0.22,$nan,$nan
1,Classification,$28.86,$10.65,$43.99
2,Review,$0.00,$nan,$nan
3,Classification,$28.86,$nan,$nan
Total,nan,$57.95,$10.65,$43.99
